In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Basics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Data
import xarray as xr
import h5py

# Helpful
import time
import datetime
import itertools
from itertools import product

# My Methods
import importlib
import CRPS
import EMOS
from CRPS import *
from EMOS import *

2023-05-16 14:47:19.793027: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/anaconda3/lib/
2023-05-16 14:47:19.793049: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


<IPython.core.display.Javascript object>

In [3]:
importlib.reload(CRPS)
importlib.reload(EMOS)

<module 'EMOS' from '/home/dchen/BA_CH_EN/EMOS.py'>

<IPython.core.display.Javascript object>

### Goal of this notebook: Implement EMOS and train it on train dataset

#### 1. Load dataset

In [29]:
t2m_all = xr.open_dataset(
    "/mnt/sda/Data2/fourcastnet/data/predictions/ensemble_2018.h5"
).isel(phony_dim_2=2, phony_dim_5=slice(1, 51))

<IPython.core.display.Javascript object>

In [30]:
t2m_all.predictions

<xarray.DataArray 'predictions' (phony_dim_0: 357, phony_dim_5: 50,
                                 phony_dim_1: 32, phony_dim_3: 120,
                                 phony_dim_4: 130)>
[8910720000 values with dtype=float32]
Dimensions without coordinates: phony_dim_0, phony_dim_5, phony_dim_1,
                                phony_dim_3, phony_dim_4

<IPython.core.display.Javascript object>

In [60]:
t2m_all.predictions.isel(
    phony_dim_0=0, phony_dim_1=1, phony_dim_3=0, phony_dim_4=0
).mean(dim="phony_dim_5")

<xarray.DataArray 'predictions' ()>
array(-0.04445939, dtype=float32)

<IPython.core.display.Javascript object>

In [61]:
t2m_X_train

<xarray.DataArray 't2m_train' (phony_dim_0: 1429, phony_dim_1: 32,
                               phony_dim_2: 120, phony_dim_3: 130,
                               phony_dim_4: 2)>
[1426713600 values with dtype=float32]
Dimensions without coordinates: phony_dim_0, phony_dim_1, phony_dim_2,
                                phony_dim_3, phony_dim_4

<IPython.core.display.Javascript object>

In [59]:
t2m_X_train[0, 1, 0, 0, 0].values

array(-1.5556693, dtype=float32)

<IPython.core.display.Javascript object>

In [34]:
# Training Dataset
t2m_train = xr.open_dataset("/Data/Delong_BA_Data/Mean_ens_std/t2m_train.h5")
t2m_X_train = t2m_train.t2m_train
t2m_y_train = t2m_train.t2m_truth

<IPython.core.display.Javascript object>

In [35]:
# Test Detaset
t2m_test = xr.open_dataset("/Data/Delong_BA_Data/Mean_ens_std/t2m_test.h5")
t2m_X_test = t2m_test.t2m_test
t2m_y_test = t2m_test.t2mtest_truth

<IPython.core.display.Javascript object>

In [36]:
t31 = t2m_X_train.isel(phony_dim_4=0, phony_dim_1=31)

<IPython.core.display.Javascript object>

In [37]:
t31[0, 0, 0].values

array(2.029154, dtype=float32)

<IPython.core.display.Javascript object>

In [38]:
ty31 = t2m_y_train.isel(phony_dim_1=31)

<IPython.core.display.Javascript object>

In [39]:
ty31[0, 0, 0].values

array(0.96273404, dtype=float32)

<IPython.core.display.Javascript object>

#### 2. Prepare Data
Make datasets for all 32 lead times

##### 2.1 Train Dataset

In [47]:
t2m_X_train_glob_mean = []
t2m_X_train_glob_std = []
t2m_y_train_glob_truth = []
for i in range(t2m_train.phony_dim_1.shape[0]):
    t2m_X_train_glob_mean.append(
        t2m_X_train.isel(phony_dim_4=0, phony_dim_1=i).values.flatten()
    )
    t2m_X_train_glob_std.append(
        t2m_X_train.isel(phony_dim_4=1, phony_dim_1=i).values.flatten()
    )
    t2m_y_train_glob_truth.append(t2m_y_train.isel(phony_dim_1=i).values.flatten())

<IPython.core.display.Javascript object>

In [ ]:
# Only necessary for Baseline, not for EMOS
# Change all zeros in std to a small value epsilon
epsilon = 1e-9  # Small epsilon to add to zeros in std
for i in range(len(t2m_X_train_glob_std)):
    t2m_X_train_glob_std[i][(t2m_X_train_glob_std[i] == 0)] += epsilon

##### 2.2 Test Dataset

In [8]:
t2m_X_test_glob_mean = []
t2m_X_test_glob_std = []
t2m_y_test_glob_truth = []
for i in range(t2m_test.phony_dim_1.shape[0]):
    t2m_X_test_glob_mean.append(
        t2m_X_test.isel(phony_dim_4=0, phony_dim_1=i).values.flatten()
    )
    t2m_X_test_glob_std.append(
        t2m_X_test.isel(phony_dim_4=1, phony_dim_1=i).values.flatten()
    )
    t2m_y_test_glob_truth.append(t2m_y_test.isel(phony_dim_1=i).values.flatten())

<IPython.core.display.Javascript object>

In [9]:
# Only necessary for Baseline, not for EMOS
# Change all zeros in std to a small value epsilon
epsilon = 1e-9  # Small epsilon to add to zeros in std
for i in range(len(t2m_X_test_glob_std)):
    t2m_X_test_glob_std[i][(t2m_X_test_glob_std[i] == 0)] += epsilon

<IPython.core.display.Javascript object>

In [62]:
# Decide on which lead_time to test
lead_time = 8

<IPython.core.display.Javascript object>

#### 3. Baseline

In [63]:
crps_baseline = crps_normal(
    mu=t2m_X_train_glob_mean[lead_time],
    sigma=t2m_X_train_glob_std[lead_time],
    y=t2m_y_train_glob_truth[lead_time],
)

<IPython.core.display.Javascript object>

In [64]:
crps_baseline.mean()

0.190077907093886

<IPython.core.display.Javascript object>

#### 4. Train global Emos

In [28]:
EMOS_glob = build_EMOS_network_keras(compile=True)

<IPython.core.display.Javascript object>

In [27]:
EMOS_glob.fit(
    [t2m_X_train_glob_mean[lead_time], t2m_X_train_glob_std[lead_time]],
    t2m_y_train_glob_truth[lead_time],
    batch_size=5000,
    epochs=5,
    validation_split=0.2,
)

NameError: name 't2m_X_train_glob_mean' is not defined

<IPython.core.display.Javascript object>

In [49]:
t2m_31_preds = EMOS_glob.predict(
    [t2m_X_test_glob_mean[lead_time], t2m_X_test_glob_std[lead_time]]
)

<IPython.core.display.Javascript object>

In [55]:
crps_emos = crps_normal(
    mu=t2m_31_preds[:, 0], sigma=t2m_31_preds[:, 0], y=t2m_y_test_glob_truth[lead_time]
)

<IPython.core.display.Javascript object>

In [56]:
crps_emos.mean()

0.11533949219491406

<IPython.core.display.Javascript object>